<img src="../images/AzPTravel_PPM.png">

# Loss components Data source

Notes from DBM

Financial Comparison

IBNR - peice bu BY IBNR%

	Visual Graph
	Losses and Claim Ratio Comparison xBU UW % overall % IBNR of IBNR Finance AY  reported view. AY/PY
	Including PY runoff vs accident year


Plan. 
- Make data set in this document export to "Results Presentation" folder

- Use export that as a Thinkcell template, 
- have visuals in ppt

    
    
Next Week
Analysis Quality Vs Trends



# Specific data and trasformations

Euro File, make separate OCR and IBNR columns for all countries

Financial File make AY/PY by BU (may have this in the P&L document)



In [1]:
commit_message = "Development and testing."
# Give a brief reason for the run.

run_control = 1
#run_type = 0 - Lite run with no reporting, not recommended.
#run_type = 1 - Lite run with normal reporting, default setting.
#run_type = 2 - Heavy run with full reporting, available for audits and troubleshooting.
#run_type = 5 - A default setting. Indicates the script is being run by an outside process without an inherited value

percode = "2021.Q1"
# Data Collection Code, this controls file paths and output names
# "-f" is the value indicating a bad inheritance from run with arg

s_format = "p"
# denotes the source data format x == Excel; j == json, p == parquet

#----------
# do not edit - this either inherits the full instance timestamp from the papermill book or captures the run time of this script.
from datetime import datetime  # datetime options
inst_datetime = datetime.now().strftime("%m%d%Y%H%M%S")

#### Notebook display options

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


#### import packages

In [3]:
#### Packages used

import os # System commands
import sys # System commands

import warnings # custom warnigns options

import glob # Directory operations
import getpass # Work with parquet
import json # Work with json

import matplotlib.pyplot as plt #Plots and Graphs
import seaborn as sns #Plots and Graphs
import numpy as np # Series and math
import pandas as pd #DataFrame and math

#excel operations
import re
import nicexcel as nl # Excel operations
import xlsxwriter # Excel operations
import openpyxl # Excel operations

#Not sure, test if needed
import timeit



#### Default Variables, these govern logic, do not edit.

In [4]:
default_dc = "20XX.QX"
default_rc = 0 #extra lite mode
dummy_perc = "33Q3" # bad inheritance

#### Script determining run context ie, manual, run_control.ipynb, or other.

In [5]:
if run_control == 5:
    run_control = default_rc 
else:
    run_control = run_control

try:
    if sys.argv[1] == "-f":
        percode = percode
    else:
        percode = sys.argv[1]

except IndexError:
    percode = default_dc
except NameError:
    percode = default_dc


if percode == "2021.Q1":
    
     prev_percode = "2020.Q4"
     prev_yearcode = "2020.Q1"

elif percode == "2020.Q4":
    
     prev_percode = "2020.Q3"
     prev_yearcode = "2019.Q4"

elif percode == "2020.Q3":
    
     prev_percode = "2020.Q1"
     prev_yearcode = "none"    

elif percode == "2020.Q1":
    
     prev_percode = "2019.Q4"
     prev_yearcode = "none"   

#### style settings

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.float_format', lambda x: '%.5f' % x)


## Begin Input Read
- Make paths used in script
- all paths should be modular and easily replaced ahead of operations.

In [7]:
#root directory
rt_path = f'\\\\hecate\\Insurance_US\\Product Development\\Product Management\\Global PPM\\Reporting\Data Collection'

# Euro file dir
fin_source_path =current_path = os.path.join(rt_path ,percode,'Finance Report')

# Source file dir
current_path = os.path.join(rt_path ,"Production" , percode)

# Output file dir
output_path = os.path.join(rt_path ,percode,'Results Presentation')  



#### Make sources used in script
- all sources should be modular and easily replaced ahead of operations.

In [8]:
fin_rep_file = os.path.join(fin_source_path,"Finance_Report.xlsx")

current_eur_file  = os.path.join(current_path  ,f'{percode}.euroconv.csv')

report_output   = os.path.join( output_path  ,f'ibnr_AY_PY_report.csv') 




#### import data

In [9]:
fin_rep_df = pd.read_excel(fin_rep_file, sheet_name = "Finance_Report")


current_eur_df = pd.read_csv(current_eur_file , engine='python') 


## Begin Transformations

-Make UW view

In [10]:
current_eur_df =current_eur_df.reset_index(drop=True)

In [11]:
current_eur_df['region'] 

0        North, Central & Eastern Europe
1        North, Central & Eastern Europe
2        North, Central & Eastern Europe
3        North, Central & Eastern Europe
4        North, Central & Eastern Europe
                      ...               
11636                      North America
11637                      North America
11638                      North America
11639                      North America
11640                      North America
Name: region, Length: 11641, dtype: object

In [12]:
current_eur_df.columns

Index(['ï»¿submission_file', 'business_unit', 'country', 'currency', 'region',
       'reporting_date_from', 'reporting_date_to', 'date_of_analysis',
       'type_of_analysis', 'analysed_months', 'reporting_year',
       'reporting_quarter', 'reporting_month', 'type_of_business',
       'type_of_account', 'distribution_type', 'lob', 'distribution_channel',
       'sub_lob', 'business_partner_name', 'business_partner_id_number',
       'product_name', 'product_id_number', 'product_family',
       'standard_product', 'number_of_products_per_row',
       'number_of_b-partners_per_row', 'number_of_policies_(written)',
       'units_of_risk_(written)', 'written_revenues_net_of_taxes',
       'written_revenues', 'number_of_policies_(earned)',
       'units_of_risk_(earned)', 'earned_revenues_net_of_taxes',
       'earned_revenues', 'earned_base_commissions', 'earned_over-commissions',
       'upfront_cash_payments', 'total_compensation',
       'number_of_claims_(paid_+_ocr_+_ibnr)', 'number

In [13]:
uw_view_cols = current_eur_df[['region', 'business_unit', 'country', 'earned_revenues_net_of_taxes',
                               'actual_incurred_losses_(paid_+_ocr_+_ibnr)', 'ocr_+_ibnr', '%_of_ibnr_on_(ocr_+_ibnr)',
                             'total_compensation' ,'total_expenses','profit_or_loss']]


#uw_view_cols.loc[0,"ibnr"] = uw_view_cols['ocr_+_ibnr'] * uw_view_cols['%_of_ibnr_on_(ocr_+_ibnr)'] #todo change to .iloc
uw_view_cols["ibnr"] = uw_view_cols['ocr_+_ibnr'] * uw_view_cols['%_of_ibnr_on_(ocr_+_ibnr)']

uw_view_cols["ocr"] = uw_view_cols['ocr_+_ibnr'] - uw_view_cols['ibnr']
uw_view_cols["non_ibnr"] = uw_view_cols['actual_incurred_losses_(paid_+_ocr_+_ibnr)'] - uw_view_cols['ibnr']




uw_view_cols

C:\Users\gharmo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\gharmo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
C:\Users\gharmo\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the do

,region,business_unit,country,earned_revenues_net_of_taxes,actual_incurred_losses_(paid_+_ocr_+_ibnr),ocr_+_ibnr,%_of_ibnr_on_(ocr_+_ibnr),total_compensation,total_expenses,profit_or_loss,ibnr,ocr,non_ibnr
0,"North, Central & Eastern Europe",AT,AT,0.00000,-49247.04000,-52275.64000,-0.00345,0.00000,0.00000,49247.04000,180.36000,-52456.00000,-49427.40000
1,"North, Central & Eastern Europe",AT,AT,0.00000,-100.00000,-100.00000,0.00000,0.00000,0.00000,100.00000,-0.00000,-100.00000,-100.00000
2,"North, Central & Eastern Europe",AT,AT,0.00000,-1100.00000,-1100.00000,0.00000,0.00000,0.00000,1100.00000,-0.00000,-1100.00000,-1100.00000
3,"North, Central & Eastern Europe",AT,AT,31328.10000,1940.94000,1142.94000,0.06293,4391.77000,8959.83660,16035.55340,71.92000,1071.02000,1869.02000
4,"North, Central & Eastern Europe",AT,AT,887.19000,-684.00000,-1084.00000,-0.01476,406.87000,253.73634,910.58366,16.00000,-1100.00000,-700.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11636,North America,US,US,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
11637,North America,US,US,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
11638,North America,US,US,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
11639,North America,US,US,9.35931,0.10210,0.10210,1.00000,3.22471,1.88037,4.15213,0.10210,0.00000,0.00000


In [14]:
uw_view_view = uw_view_cols.groupby(['region','business_unit']).sum()

uw_view_view = uw_view_view / 1000


uw_view_view['%_of_ibnr_on_(ocr_+_ibnr)'] = uw_view_view['%_of_ibnr_on_(ocr_+_ibnr)'] * 1000

uw_view_view = uw_view_view.reset_index()

# #uw_view_view = uw_view_view['region'].ffill(inplace=True)


uw_view_view

,region,business_unit,earned_revenues_net_of_taxes,actual_incurred_losses_(paid_+_ocr_+_ibnr),ocr_+_ibnr,%_of_ibnr_on_(ocr_+_ibnr),total_compensation,total_expenses,profit_or_loss,ibnr,ocr,non_ibnr
0,APAC,AU,9332.57291,1811.87266,1566.89501,15.11334,216.12795,4832.42124,2472.15106,296.84186,1270.05316,1515.03080
1,North America,CA,7906.94835,4417.08028,2695.31852,-5.92243,2233.72250,1771.53325,-515.38769,-938.98977,3634.30829,5356.07005
2,North America,US,99445.31013,15936.43673,6999.37296,8306.09717,51631.83525,19394.67931,12482.35884,1973.47479,5025.89817,13962.96194
3,"North, Central & Eastern Europe",AT,3972.80009,547.22522,-555.61830,-17.68921,1033.95227,1151.84376,1239.77885,53.39126,-609.00956,493.83396
4,"North, Central & Eastern Europe",CH,7506.57124,1362.27409,685.52468,46.56918,1856.36679,2101.87606,2186.05430,489.94799,195.57670,872.32610
5,"North, Central & Eastern Europe",CZ,387.39204,274.98698,0.00000,0.00000,173.00934,59.40792,-120.01221,0.00000,0.00000,274.98698
6,"North, Central & Eastern Europe",DE,17442.45767,6578.57127,-585.05130,-117.17776,4520.19157,4116.42001,2227.27482,2744.41142,-3329.46272,3834.15985
7,"North, Central & Eastern Europe",GR,109.62252,19.89267,18.84584,0.00000,42.33664,24.82346,22.56975,0.00000,18.84584,19.89267
8,"North, Central & Eastern Europe",PL,3802.68099,1314.10645,403.20276,2.20180,1579.91999,1071.12415,-162.46961,83.72280,319.47996,1230.38365
9,"Western Europe, LATAM & MEA",ES,736.24962,148.35317,127.31118,0.00000,339.08099,284.14694,-35.33148,0.00000,127.31118,148.35317


-make fin view

In [15]:
fin_rep_df.columns

Index(['DC_BU_Code', 'BU_Desc_ Finance', 'BU_Desc', 'Total SGW', 'Total NER',
       'Commissions Acquis in %', 'Claims Costs (LAE & FA) in %',
       'FA Acquis in %', 'FA Acquis in % -Second', 'Non manageable in %',
       'Loss a.y', 'Loss ratio in %', 'Total Combined ratio in %',
       'Technical Result', 'Operating Profit', 'Combined Ratio Calculation',
       'Check'],
      dtype='object')

In [16]:
fin_rep_view = fin_rep_df[['DC_BU_Code', 'BU_Desc_ Finance', 'BU_Desc', 'Total NER', 'Loss a.y', 'Loss ratio in %',
            'Commissions Acquis in %', 'Claims Costs (LAE & FA) in %', 'FA Acquis in %', 
            'FA Acquis in % -Second', 'Non manageable in %',    'Total Combined ratio in %' , 'Technical Result' ]]

fin_rep_view['Losses AY - Euro'] = fin_rep_view['Loss a.y'] * fin_rep_view['Total NER']

fin_rep_view['Losses - Euro'] = fin_rep_view['Loss ratio in %'] * fin_rep_view['Total NER']

fin_rep_view['Losses PY - Euro'] = fin_rep_view['Losses - Euro']  - fin_rep_view['Losses AY - Euro'] 


fin_rep_view['Expense - Euro'] =  (fin_rep_view['Claims Costs (LAE & FA) in %'] + 
                                       fin_rep_view['FA Acquis in %'] +  fin_rep_view['FA Acquis in % -Second'] +
                                        fin_rep_view['Non manageable in %'] ) * fin_rep_view['Total NER']



fin_rep_view['Commissions - Euro'] = fin_rep_view['Commissions Acquis in %'] * fin_rep_view['Total NER']


for c in fin_rep_view:
    
    if  np.issubdtype(fin_rep_view[c].dtype, np.number):

        fin_rep_view[c] = fin_rep_view[c] 
    else:
        pass

    
fin_rep_view

C:\Users\gharmo\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\gharmo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\Users\gharmo\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

,DC_BU_Code,BU_Desc_ Finance,BU_Desc,Total NER,Loss a.y,Loss ratio in %,Commissions Acquis in %,Claims Costs (LAE & FA) in %,FA Acquis in %,FA Acquis in % -Second,Non manageable in %,Total Combined ratio in %,Technical Result,Losses AY - Euro,Losses - Euro,Losses PY - Euro,Expense - Euro,Commissions - Euro
0,AU,BU Australia,Australia,6328.70000,0.31762,0.19540,0.23989,0.23837,0.11199,0.22651,-0.01024,1.00193,-12.20000,2010.13000,1236.63000,-773.50000,3586.07000,1518.20000
1,CN,BU China,China,203.73000,0.09449,-0.27635,0.41108,3.48368,1.33535,1.02346,-0.31061,5.66662,-950.73000,19.25000,-56.30000,-75.55000,1127.01000,83.75000
2,NaN,BU Indonesia,Indonesia,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
3,NaN,BU India,India,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
4,NaN,BU Japan,Japan,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
5,NaN,BU Korea,Korea,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
6,MY,BU Malaysia,Malaysia,6.93000,0.00000,0.00000,0.55844,8.57143,1.34055,2.90043,0.00000,13.37085,-85.73000,0.00000,0.00000,0.00000,88.79000,3.87000
7,NZ,BU New Zealand,New Zealand,727.09000,1.90025,3.77750,0.29585,0.17999,0.27024,0.35305,0.03282,4.90945,-2842.52000,1381.65000,2746.58000,1364.93000,607.92000,215.11000
8,SG,BU Singapore,Singapore,830.24000,0.01819,0.02328,0.07545,0.08767,0.16561,0.16470,0.27163,0.78835,175.72000,15.10000,19.33000,4.23000,572.55000,62.64000
9,NaN,BU Thailand,Thailand,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000


- Merge 2 frames

In [17]:
uw_view_view

,region,business_unit,earned_revenues_net_of_taxes,actual_incurred_losses_(paid_+_ocr_+_ibnr),ocr_+_ibnr,%_of_ibnr_on_(ocr_+_ibnr),total_compensation,total_expenses,profit_or_loss,ibnr,ocr,non_ibnr
0,APAC,AU,9332.57291,1811.87266,1566.89501,15.11334,216.12795,4832.42124,2472.15106,296.84186,1270.05316,1515.03080
1,North America,CA,7906.94835,4417.08028,2695.31852,-5.92243,2233.72250,1771.53325,-515.38769,-938.98977,3634.30829,5356.07005
2,North America,US,99445.31013,15936.43673,6999.37296,8306.09717,51631.83525,19394.67931,12482.35884,1973.47479,5025.89817,13962.96194
3,"North, Central & Eastern Europe",AT,3972.80009,547.22522,-555.61830,-17.68921,1033.95227,1151.84376,1239.77885,53.39126,-609.00956,493.83396
4,"North, Central & Eastern Europe",CH,7506.57124,1362.27409,685.52468,46.56918,1856.36679,2101.87606,2186.05430,489.94799,195.57670,872.32610
5,"North, Central & Eastern Europe",CZ,387.39204,274.98698,0.00000,0.00000,173.00934,59.40792,-120.01221,0.00000,0.00000,274.98698
6,"North, Central & Eastern Europe",DE,17442.45767,6578.57127,-585.05130,-117.17776,4520.19157,4116.42001,2227.27482,2744.41142,-3329.46272,3834.15985
7,"North, Central & Eastern Europe",GR,109.62252,19.89267,18.84584,0.00000,42.33664,24.82346,22.56975,0.00000,18.84584,19.89267
8,"North, Central & Eastern Europe",PL,3802.68099,1314.10645,403.20276,2.20180,1579.91999,1071.12415,-162.46961,83.72280,319.47996,1230.38365
9,"Western Europe, LATAM & MEA",ES,736.24962,148.35317,127.31118,0.00000,339.08099,284.14694,-35.33148,0.00000,127.31118,148.35317


In [18]:
reporting_view = uw_view_view.merge( fin_rep_view ,how='left', left_on='business_unit', right_on='DC_BU_Code')

reporting_view["Gap"] =  reporting_view['actual_incurred_losses_(paid_+_ocr_+_ibnr)'] -  reporting_view['Losses - Euro'] 

reporting_view

,region,business_unit,earned_revenues_net_of_taxes,actual_incurred_losses_(paid_+_ocr_+_ibnr),ocr_+_ibnr,%_of_ibnr_on_(ocr_+_ibnr),total_compensation,total_expenses,profit_or_loss,ibnr,...,FA Acquis in % -Second,Non manageable in %,Total Combined ratio in %,Technical Result,Losses AY - Euro,Losses - Euro,Losses PY - Euro,Expense - Euro,Commissions - Euro,Gap
0,APAC,AU,9332.57291,1811.87266,1566.89501,15.11334,216.12795,4832.42124,2472.15106,296.84186,...,0.22651,-0.01024,1.00193,-12.20000,2010.13000,1236.63000,-773.50000,3586.07000,1518.20000,575.24266
1,North America,CA,7906.94835,4417.08028,2695.31852,-5.92243,2233.72250,1771.53325,-515.38769,-938.98977,...,0.06965,-0.01596,1.06075,-963.83000,2852.80000,1125.05000,-1727.75000,8694.68000,7010.09000,3292.03028
2,North America,US,99445.31013,15936.43673,6999.37296,8306.09717,51631.83525,19394.67931,12482.35884,1973.47479,...,0.09859,0.03063,1.03887,-3820.69000,18061.16000,18450.96000,389.80000,36827.85000,46845.15000,-2514.52327
3,"North, Central & Eastern Europe",AT,3972.80009,547.22522,-555.61830,-17.68921,1033.95227,1151.84376,1239.77885,53.39126,...,0.10451,-0.00538,0.01837,3961.77000,1450.14000,-519.00000,-1969.14000,1327.00000,-733.85000,1066.22522
4,"North, Central & Eastern Europe",CH,7506.57124,1362.27409,685.52468,46.56918,1856.36679,2101.87606,2186.05430,489.94799,...,0.14043,0.00529,1.04796,-282.29000,2112.35000,807.39000,-1304.96000,3196.48000,2164.57000,554.88409
5,"North, Central & Eastern Europe",CZ,387.39204,274.98698,0.00000,0.00000,173.00934,59.40792,-120.01221,0.00000,...,0.01142,-0.01845,-1.16208,808.38000,222.03000,-664.53000,-886.56000,64.11000,165.93000,939.51698
6,"North, Central & Eastern Europe",DE,17442.45767,6578.57127,-585.05130,-117.17776,4520.19157,4116.42001,2227.27482,2744.41142,...,0.16605,-0.03866,1.03050,-374.00000,4967.00000,3964.00000,-1003.00000,5185.00000,3486.00000,2614.57127
7,"North, Central & Eastern Europe",GR,109.62252,19.89267,18.84584,0.00000,42.33664,24.82346,22.56975,0.00000,...,0.09259,0.00000,0.87037,14.00000,17.00000,-32.00000,-49.00000,72.00000,54.00000,51.89267
8,"North, Central & Eastern Europe",PL,3802.68099,1314.10645,403.20276,2.20180,1579.91999,1071.12415,-162.46961,83.72280,...,0.08089,-0.00085,1.16094,-487.76000,1346.89000,1159.88000,-187.01000,1071.40000,1287.23000,154.22645
9,"Western Europe, LATAM & MEA",ES,736.24962,148.35317,127.31118,0.00000,339.08099,284.14694,-35.33148,0.00000,...,0.10507,0.00000,0.97117,22.87000,273.38000,-279.95000,-553.33000,674.48000,375.91000,428.30317


- export to csv

In [19]:
reporting_view.to_csv(report_output, ',', index=False, encoding='utf-8-sig')